In [1]:
from src.api import ModelAPI  
ModelAPI.Trainer.available_modules()

{'lgbm': src.res.algo.boost.booster.lgbm.Lgbm,
 'ada': src.res.algo.boost.booster.ada.AdaBoost,
 'xgboost': src.res.algo.boost.booster.xgboost.XgBoost,
 'catboost': src.res.algo.boost.booster.catboost.CatBoost,
 'simple_lstm': src.res.algo.nn.model.Recurrent.simple_lstm,
 'gru': src.res.algo.nn.model.Recurrent.gru,
 'lstm': src.res.algo.nn.model.Recurrent.lstm,
 'resnet_lstm': src.res.algo.nn.model.Recurrent.resnet_lstm,
 'resnet_gru': src.res.algo.nn.model.Recurrent.resnet_gru,
 'transformer': src.res.algo.nn.model.Recurrent.transformer,
 'tcn': src.res.algo.nn.model.Recurrent.tcn,
 'rnn_ntask': src.res.algo.nn.model.Recurrent.rnn_ntask,
 'rnn_general': src.res.algo.nn.model.Recurrent.rnn_general,
 'gru_dsize': src.res.algo.nn.model.Recurrent.gru_dsize,
 'patch_tst': src.res.algo.nn.model.PatchTST.patch_tst,
 'modern_tcn': src.res.algo.nn.model.ModernTCN.modern_tcn,
 'ts_mixer': src.res.algo.nn.model.TSMixer.ts_mixer,
 'tra': src.res.algo.nn.model.TRA.tra,
 'factor_vae': src.res.algo.

In [1]:
from src.api import ModelAPI  
app = ModelAPI.train_model(module = 'gru' , short_test = True)

ModuleNotFoundError: No module named 'src.res.algo.nn.util'

In [11]:
from src.proj import Proj
[cls.__qualname__ for cls in Proj.States.trainer.model.__class__.__mro__]

['NNPredictor',
 'BasePredictorModel',
 'ModelStreamLineWithTrainer',
 'ModelStreamLine',
 'ABC',
 'object']

In [7]:
from src.proj import Proj
self = Proj.States.trainer.metrics.model_metrics

dfs = {}
for name in self.metric_names:
    dfs[name] = self.get_table(name)
dfs

KeyError: 'None of [None] are in the columns'

In [9]:
self.tables['test_scores']

[Empty DataFrame
 Columns: []
 Index: []]

In [ ]:
import torch

def loss(label : torch.Tensor , pred : torch.Tensor | None = None , w : torch.Tensor | None = None , dim = None , 
         quantiles : list[float] = [0.1,0.5,0.9] , predictions : torch.Tensor | None = None , **kwargs):
    assert predictions is not None , f'predictions should be provided'
    assert predictions.shape[-1] == len(quantiles) , f'shape of predictions {predictions.shape} should be (...,{len(quantiles)})'
    if predictions.ndim == label.ndim + 1: 
        predictions = predictions.squeeze(-2)
    assert predictions.ndim == label.ndim == 2 , f'shape of predictions {predictions.shape} and label {label.shape} should be (...,1)'
    if w is None:
        w1 = 1.
    else:
        w1 = w / w.sum(dim=dim,keepdim=True) * (w.numel() if dim is None else w.size(dim=dim))
    
    losses = []
    label = label.expand_as(predictions)
    for i, q in enumerate(quantiles):
        pred_q = predictions[..., i:i+1]
        error = label - pred_q
        valid = ~error.isnan()
        loss = torch.max(q * error[valid], (q - 1) * error[valid])
        print(loss)
        losses.append((w1 * loss).mean(dim=dim,keepdim=True))
    print(losses)
    v = torch.stack(losses,dim=-1).mean(dim=-1)
    return v


tensor([0.2509, 0.2509, 0.2509,  ..., 0.6996, 0.6996, 0.6996], device='mps:0')
tensor([0.0764, 0.0764, 0.0764,  ..., 0.2467, 0.2467, 0.2467], device='mps:0')
tensor([0.0659, 0.0659, 0.0659,  ..., 0.1139, 0.1139, 0.1139], device='mps:0')
[tensor([0.4907], device='mps:0'), tensor([0.3703], device='mps:0'), tensor([0.2238], device='mps:0')]


tensor([0.3616], device='mps:0')

In [1]:
import torch
import gc


def test_operations(iterations: int, shape: tuple[int, int]) -> None:
    print(f"PyTorch version: {torch.__version__}")
    # Test 1: torch.unique
    print("\nTest 1: torch.unique")
    x = torch.randint(0, 2, shape, device="mps")
    for i in range(iterations):
        y = torch.unique(x)
        del y

        # Empty cache and collect garbage to make sure
        torch.mps.empty_cache()
        gc.collect()

        if i % 10 == 0:
            print(
                f"Iter {i}: Driver Allocated Memory: {torch.mps.driver_allocated_memory() / (1024**2):.2f}MB, Current Allocated Memory: {torch.mps.current_allocated_memory() / (1024**2):.2f}MB"
            )

    # Test 2: torch.sort (comparison)
    print("\nTest 2: torch.sort")
    for i in range(iterations):
        y = torch.sort(x)[0]
        del y
        # Empty cache and collect garbage to make sure
        torch.mps.empty_cache()
        gc.collect()

        if i % 10 == 0:
            print(
                f"Iter {i}: Driver memory: {torch.mps.driver_allocated_memory() / (1024**2):.2f}MB, Current memory: {torch.mps.current_allocated_memory() / (1024**2):.2f}MB"
            )


test_operations(iterations=100, shape=(2000, 10))

PyTorch version: 2.7.1

Test 1: torch.unique
Iter 0: Driver Allocated Memory: 10.73MB, Current Allocated Memory: 0.15MB
Iter 10: Driver Allocated Memory: 10.73MB, Current Allocated Memory: 0.15MB
Iter 20: Driver Allocated Memory: 10.73MB, Current Allocated Memory: 0.15MB
Iter 30: Driver Allocated Memory: 10.72MB, Current Allocated Memory: 0.15MB
Iter 40: Driver Allocated Memory: 10.72MB, Current Allocated Memory: 0.15MB
Iter 50: Driver Allocated Memory: 10.72MB, Current Allocated Memory: 0.15MB
Iter 60: Driver Allocated Memory: 10.72MB, Current Allocated Memory: 0.15MB
Iter 70: Driver Allocated Memory: 10.72MB, Current Allocated Memory: 0.15MB
Iter 80: Driver Allocated Memory: 10.72MB, Current Allocated Memory: 0.15MB
Iter 90: Driver Allocated Memory: 10.72MB, Current Allocated Memory: 0.15MB

Test 2: torch.sort
Iter 0: Driver memory: 10.72MB, Current memory: 0.15MB
Iter 10: Driver memory: 10.72MB, Current memory: 0.15MB
Iter 20: Driver memory: 10.72MB, Current memory: 0.15MB
Iter 30: 

In [ ]:
from src.api import ModelAPI
ModelAPI.clear_st_models()

Deleting lgbm_day_ShortTest in /Users/mengkjin/workspace/learndl/models


In [ ]:
from src.proj import Proj
Proj.States

InstanceRecord(names=['trainer', 'account'])

In [ ]:
from src.api import ModelAPI
ModelAPI.Extractor.update()



model_name is None, update all hidden models
{'name': 'gru_day', 'nums': None, 'submodels': ['best']}
Beware! Should be at server or short_test, but short_test is False now!
try using d:\Coding\learndl\learndl\data\DataSet/day.20240607.pt , success!
Load  2 DataBlocks...... finished! Cost 0.08 secs
Align 2 DataBlocks...... finished! Cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


100%|██████████| 316/316 [00:00<00:00, 3047.38it/s]

--------------------------------------------------------------------------------


In [ ]:
from src.api import ModelAPI

ModelAPI.update_models()

src.basic.INSTANCE_RECORD can be accessed to check ['trainer', 'account']
Basic module imported!
Use device name: NVIDIA GeForce RTX 4090


24-12-15 18:40:32|MOD:display     |: Model Specifics:
24-12-15 18:40:32|MOD:display     |: Start Process [Data] at Sun Dec 15 18:40:32 2024!


--------------------------------------------------------------------------------
**************************** RECONSTRUCT TRAIN DATA ****************************
predict is False , Data Processing start!
6 datas : ['y', 'day', '30m', 'style', 'indus', 'week'] , from 20070101 to None
y is up to 20241215121914 already!
day is up to 20241215122636 already!
30m is up to 20241215122719 already!
style is up to 20241215124444 already!
indus is up to 20241215124618 already!
week is up to 20241215124746 already!
Data Processing Finished! Cost 0.00 Seconds
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************** UPDATE MODELS ********************************
--------------------------------------------------------------------------------
***************************** PARSER TRAINING ARGS *****************************
--Process Queue : Data + Fit + Test
--Confi

24-12-15 18:40:36|MOD:display     |: Finish Process [Data], Cost 3.7 Secs
24-12-15 18:40:36|MOD:display     |: Start Process [Fit] at Sun Dec 15 18:40:36 2024!
24-12-15 18:40:36|MOD:display     |: Finish Process [Fit], Cost 0.0 Secs
24-12-15 18:40:36|MOD:display     |: Start Process [Test] at Sun Dec 15 18:40:36 2024!
24-12-15 18:40:36|MOD:display     |: Testing Mean Score(spearman):
24-12-15 18:40:36|MOD:display     |: Finish Process [Test], Cost 0.0 Secs
24-12-15 18:40:36|MOD:timer       |: Main Process Finished! Cost 3.8 Seconds
24-12-15 18:40:36|MOD:display     |: Model Specifics:
24-12-15 18:40:36|MOD:display     |: Start Process [Data] at Sun Dec 15 18:40:36 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
--------------------------------------------------------------------------------
***************************** PARSER TRAINING ARGS *****************************
--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gruRTN_day!
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************* SETUP CALLBACKS *******************************
ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
CallbackTimer(verbosity=2) , record time cost of callback hooks
EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 1

24-12-15 18:40:40|MOD:display     |: Finish Process [Data], Cost 3.7 Secs
24-12-15 18:40:40|MOD:display     |: Start Process [Fit] at Sun Dec 15 18:40:40 2024!
24-12-15 18:40:40|MOD:display     |: Finish Process [Fit], Cost 0.0 Secs
24-12-15 18:40:40|MOD:display     |: Start Process [Test] at Sun Dec 15 18:40:40 2024!
24-12-15 18:40:40|MOD:display     |: Testing Mean Score(spearman):
24-12-15 18:40:40|MOD:display     |: Finish Process [Test], Cost 0.0 Secs
24-12-15 18:40:40|MOD:timer       |: Main Process Finished! Cost 3.7 Seconds
24-12-15 18:40:40|MOD:display     |: Model Specifics:
24-12-15 18:40:40|MOD:display     |: Start Process [Data] at Sun Dec 15 18:40:40 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
--------------------------------------------------------------------------------
***************************** PARSER TRAINING ARGS *****************************
--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gru_avg!
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************* SETUP CALLBACKS *******************************
ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
CallbackTimer(verbosity=2) , record time cost of callback hooks
EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100,

24-12-15 18:40:44|MOD:display     |: Finish Process [Data], Cost 3.7 Secs
24-12-15 18:40:44|MOD:display     |: Start Process [Fit] at Sun Dec 15 18:40:44 2024!
24-12-15 18:40:44|MOD:display     |: Finish Process [Fit], Cost 0.0 Secs
24-12-15 18:40:44|MOD:display     |: Start Process [Test] at Sun Dec 15 18:40:44 2024!
24-12-15 18:40:44|MOD:display     |: Testing Mean Score(spearman):
24-12-15 18:40:44|MOD:display     |: Finish Process [Test], Cost 0.0 Secs
24-12-15 18:40:44|MOD:timer       |: Main Process Finished! Cost 3.7 Seconds


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
--------------------------------------------------------------------------------
